In [1]:
!pip install nltk

In [2]:
import pandas as pd
import random
from azureml.core import Workspace, Datastore, Dataset
from nltk.tokenize import WhitespaceTokenizer


In [28]:
df = pd.read_csv("~/cloudfiles/code/Users/capstonesipsml/Data/Alpha_Dataset.csv", names=["Data", "Label"], header=0)

usernames_df = pd.read_csv("~/cloudfiles/code/Users/mitchall.boesel/Twitter/tweet_tokens_cleaned.csv", names=["Data", "Label"], header=0)
hostnames_df = pd.read_csv("~/cloudfiles/code/Users/caesar.white/hostnames_strings.csv", names=["Data", "Label"], header=0)
filenames_df = pd.read_csv("~/cloudfiles/code/Users/jongyun.kim/featuriaztion/filenames.csv", names=["Data", "Label"], header=0)
ip_df = pd.read_csv("~/cloudfiles/code/Users/cougsInAzure/IP_Datapoints.csv", names=["Data", "Label"], header=0)
servernames_df = pd.read_csv("~/cloudfiles/code/Users/mitchall.boesel/servernames.csv", names=["Data", "Label"], header=0)

In [29]:
# Importing everyone's datasets

#usernames_df = pd.read_csv("../..//Dataset_CSVs/tweetsv2.csv", names=["Data", "Label"], header=0)
usernames_df = pd.read_csv("~/cloudfiles/code/Users/mitchall.boesel/Twitter/tweet_tokens_cleaned.csv", names=["Data", "Label"], header=0)
hostnames_df = pd.read_csv("../../Dataset_CSVs/hostname_stringsv2.csv", names=["Data", "Label"], header=0)
filenames_df = pd.read_csv("../../Dataset_CSVs/filenamesv2.csv", names=["Data", "Label"], header=0)
ip_df = pd.read_csv("../../Dataset_CSVs/ip_datapointsv2.csv", names=["Data", "Label"], header=0)
servernames_df = pd.read_csv("../../Dataset_CSVs/servernamesv2.csv", names=["Data", "Label"], header=0)


In [30]:
usernames_df['Label'].value_counts()

nothing     10000
username     3871
Name: Label, dtype: int64

In [31]:
l = [usernames_df,hostnames_df,filenames_df,ip_df,servernames_df]
for d in l:
    print(d['Label'].value_counts())

nothing     10000
username     3871
Name: Label, dtype: int64
hostname    5025
nothing     4975
Name: Label, dtype: int64
filename    8741
Name: Label, dtype: int64
IP         2527
nothing    2473
Name: Label, dtype: int64
servername    186
Name: Label, dtype: int64


In [32]:
# Combining datasets and randomizing resulting list

dataset = usernames_df.values.tolist() + hostnames_df.values.tolist() + filenames_df.values.tolist() + ip_df.values.tolist() + servernames_df.values.tolist()
random.shuffle(dataset)
print(len(dataset))


37798


In [ ]:
data_df = pd.DataFrame(dataset, columns=["Data","Label"])
data_df.head()
data_df.to_csv("~/cloudfiles/code/Users/capstonesipsml/Data/NonFeaturized_Data.csv", index=False)

In [38]:
# Features

# Username specific features
def LowerUnderscoreUpper(s):
    l = len(s)

    for i in range(0,l-2):
        if s[i].islower() and s[i+1] == '_' and s[i+2].isupper():
            return True
    return False

def HasUnderscore(s):
    s = str(s)

    for c in s:
        if c == '_':
            return True
    return False

def LowerUpperLower(s):
    s = str(s)

    for i in range(0, len(s)-2):
        if s[i].islower() and s[i+1].isupper() and s[i+2].islower():
            return True
    return False

def MultipleLowerUpperLower(s):
    s = str(s)

    flag = False
    for i in range(0, len(s)-2):
        if s[i].islower() and s[i+1].isupper() and s[i+2].islower():
            if flag:
                return True
            else:
                flag = True
    return False

def ExactlyTwoUppercase(s):
    count = 0

    for c in s:
        if c.isupper():
            count+=1
    if count ==2:
        return True
    else:
        return False

def AllLowerMoreThan(bound, s):
    s = str(s)

    if len(s) < bound:
        return False
    for c in s:
        if c.isupper():
            return False
    return True

def AdjacentUppers(s):
    s = str(s)

    for i in range(0,len(s)-1):
        if s[i].isupper() and s[i+1].isupper():
            return True
    return False

def StartLetterEndNonLetter(s):
    s = str(s)

    if s[0].isalpha() and not s[-1].isalpha():
        return True
    return False

def IsUrl(s):
    s = str(s)
    l = len(s)
    if l > 3:
        if s[0:4] == "http":
            return True
    if l > 4:
        if s[0:5] == "https":
            return True
    return False

# Hostname specific features
def LengthGTLT(gt, ls, s):
    l = len(str(s))
    if l >= gt and l <= ls:
        return True
    else:
        return False

def IllegalHostnameChars(s):
    s = str(s)
    illegal_chars = [".", "\", ""/", "*", "?", "\"", "<", ">", "|", ",", "~", ":", "!", "@", "#", "$", "%", "^", "&", "'", "(", ")", "{", "}", " "]
    for c in s:
        if c in illegal_chars:
            return True
    return False

def AlphaOrDigit(s):
    s = str(s)
    if s.isalpha() or s.isdigit():
        return True
    return False

def HostIllegalEnding(s):
    s = str(s)
    if s[-1] == '-' or s[-1] == '.':
        return True
    return False

# Filename specific features

def ContainsPeriod(s):
    s = str(s)
    for c in s:
        if c == '.':
            return True
    return False


def HasSlash(s):
    s = str(s)
    for c in s:
        if c == '/' or c == '\\':
            return True
    return False

def HasMultipleSlash(s):
    s = str(s)
    slashes = ['/','\\']
    flag = False
    for c in s:
        if not flag and c in slashes:
            flag = True
        elif flag and c in slashes:
            return True
    return False

def HasPossibleExtension(s):
    s = str(s)
    for i in range(0,len(s)):
        if s[i] == '.':
            x = s[i:-1]
            if len(x) >= 2 and len(x) <= 4:
                return True
    return False

# IP Features
def NumbersThenPeriod(s):
    s = str(s)
    freq = 0
    for i in range(0,len(s)-1):
        if s[i] >= '0' and s[i] <= '9' and s[i+1] == '.':
            freq += 1
    return True if freq == 3 else False

def AtLeastFourDigits(s):
    s = str(s)
    counter = 0
    for c in s:
        if c.isnumeric():
            counter += 1
    return True if counter >=4 else False

# Servername Features

def HasPeriodAndSlash(s):
    s = str(s)
    period = False
    slash = False
    for c in s:
        if c == '.':
            period = True
        if s =='/':
            slash = True
    return True if period and slash else False

def HasInstanceNumPeriod(s):
    s = str(s)
    for i in range(0,len(s)-1):
        if s[i].isnumeric() and s[i+1] == '.':
            return True
    return False

def HasMultipleNumPeriod(s):
    s = str(s)
    flag = False
    for i in range(0,len(s)-1):
        if s[i].isnumeric() and s[i+1] == '.' and not flag:
            flag = True
        if s[i].isnumeric() and s[i+1] == '.' and flag:
            return True
    return False

# List of the feature names, for resulting csv file column names

feature_names =["LowerUnderscoreUpper", "HasUnderscore", "LowerUpperLower", "MultipleLowerUpperLower", "ExactlyTwoUppercase", "AllLowerMoreThan",
    "AdjacentUppers", "StartLetterEndNonLetter", "LengthGTLT", "IllegalHostnameChars", "AlphaOrDigit", "HostIllegalEnding", "ContainsPeriod",
    "HasSlash", "HasMultipleSlash", "HasPossibleExtension", "NumbersThenPeriod", "AtLeastFourDigits", "HasPeriodAndSlash", "HasInstanceNumPeriod", 
    "HasMultipleNumPeriod", "Label"]
'''
feature_names =["LowerUnderscoreUpper", "HasUnderscore", "LowerUpperLower", "MultipleLowerUpperLower", "ExactlyTwoUppercase", "AllLowerMoreThan",
    "AdjacentUppers", "LengthGTLT", "IllegalHostnameChars", "AlphaOrDigit", "HostIllegalEnding", "ContainsPeriod",
    "HasSlash", "HasMultipleSlash", "HasPossibleExtension", "NumbersThenPeriod", "AtLeastFourDigits", "HasPeriodAndSlash", "HasInstanceNumPeriod", 
    "HasMultipleNumPeriod", "Label"]
'''

'\nfeature_names =["LowerUnderscoreUpper", "HasUnderscore", "LowerUpperLower", "MultipleLowerUpperLower", "ExactlyTwoUppercase", "AllLowerMoreThan",\n    "AdjacentUppers", "LengthGTLT", "IllegalHostnameChars", "AlphaOrDigit", "HostIllegalEnding", "ContainsPeriod",\n    "HasSlash", "HasMultipleSlash", "HasPossibleExtension", "NumbersThenPeriod", "AtLeastFourDigits", "HasPeriodAndSlash", "HasInstanceNumPeriod", \n    "HasMultipleNumPeriod", "Label"]\n'

In [10]:
# extracting features and labels for strings
feature_list = []
 
tk = WhitespaceTokenizer() 

for dp in dataset:
    d = str(dp[0])
    label = dp[1]
    tokens = tk.tokenize(d)
    tokens = list(filter(lambda x: not IsUrl(x), tokens))
    dp_list = []
    dp_list.append(True) if True in list(map(lambda x: LowerUnderscoreUpper(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasUnderscore(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: LowerUpperLower(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: MultipleLowerUpperLower(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: ExactlyTwoUppercase(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: AllLowerMoreThan(10,x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: AdjacentUppers(x), tokens)) else dp_list.append(False)
    #dp_list.append(True) if True in list(map(lambda x: StartLetterEndNonLetter(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: LengthGTLT(1, 15, x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: IllegalHostnameChars(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: AlphaOrDigit(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HostIllegalEnding(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: ContainsPeriod(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasSlash(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasMultipleSlash(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasPossibleExtension(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: NumbersThenPeriod(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: AtLeastFourDigits(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasPeriodAndSlash(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasInstanceNumPeriod(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasMultipleNumPeriod(x), tokens)) else dp_list.append(False)
    dp_list.append(label)
    feature_list.append(dp_list)


In [39]:
# extracting features and labels for tokens
feature_list = []
 
for dp in dataset:
    if IsUrl(dp):
        continue
    x = str(dp[0])
    label = dp[1]
    dp_list = []
    dp_list.append(LowerUnderscoreUpper(x))
    dp_list.append(HasUnderscore(x))
    dp_list.append(LowerUpperLower(x))
    dp_list.append(MultipleLowerUpperLower(x))
    dp_list.append(ExactlyTwoUppercase(x))
    dp_list.append(AllLowerMoreThan(10,x))
    dp_list.append(AdjacentUppers(x))
    dp_list.append(StartLetterEndNonLetter(x))
    dp_list.append(LengthGTLT(1, 15, x))
    dp_list.append(IllegalHostnameChars(x))
    dp_list.append(AlphaOrDigit(x))
    dp_list.append(HostIllegalEnding(x))
    dp_list.append(ContainsPeriod(x))
    dp_list.append(HasSlash(x))
    dp_list.append(HasMultipleSlash(x))
    dp_list.append(HasPossibleExtension(x))
    dp_list.append(NumbersThenPeriod(x))
    dp_list.append(AtLeastFourDigits(x))
    dp_list.append(HasPeriodAndSlash(x))
    dp_list.append(HasInstanceNumPeriod(x))
    dp_list.append(HasMultipleNumPeriod(x))
    dp_list.append(label)
    feature_list.append(dp_list)


In [40]:
# Writing featurized + labeled dataset to a csv file

labeled_features_df = pd.DataFrame(feature_list, columns=feature_names)
labeled_features_df.head()
labeled_features_df.to_csv("~/cloudfiles/code/Users/capstonesipsml/Data/token_features.csv", index=False)

In [50]:
import os


In [65]:
# Registering the featurized + labeled dataset
print(os.getcwd())
subscription_id = '9d0dfa04-d2f8-4521-b945-b3a7dbf43946'
resource_group = 'CougsInAzure'
workspace_name = 'CougsInAzure2'

workspace = Workspace(subscription_id, resource_group, workspace_name)
datastore = workspace.get_default_datastore()
datastore.upload(src_dir="Data/", target_path="Data.",overwrite=True)
alpha_featurized_ds = Dataset.Tabular.from_delimited_files(path=[(datastore, ("./token_features.csv"))])

featurized_registered_dataset = alpha_featurized_ds.register(workspace=workspace,
                                 name='Tokens_Featurized',
                                 description='Tokens instead of strings')
#~/cloudfiles/code/

/mnt/batch/tasks/shared/LS_root/mounts/clusters/mitchall-compute/code/Users/capstonesipsml/Data
Uploading an estimated of 42 files
Uploading ./.amlignore
Uploading ./.amlignore.amltmp
Uploading ./alpha_dataset-copy.csv.amltmp
Uploading ./Alpha_Dataset.csv
Uploading ./Alpha_Featurized.csv
Uploading ./create_alpha_dataset-copy.ipynb.amltmp
Uploading ./Create_Alpha_Dataset.ipynb
Uploading ./create_alpha_dataset.ipynb.amltmp
Uploading ./data_statistics-copy.ipynb.amltmp
Uploading ./Data_Statistics.ipynb
Uploading ./data_statistics.ipynb.amltmp
Uploading ./feature_testing.ipynb.amltmp
Uploading ./token_features.csv
Uploading ./.ipynb_aml_checkpoints/Create_Alpha_Dataset-checkpoint2021-0-29-9-8-24.ipynb
Uploading ./.ipynb_aml_checkpoints/Create_Alpha_Dataset-checkpoint2021-0-30-15-51-46.ipynb
Uploading ./.ipynb_aml_checkpoints/Create_Alpha_Dataset-checkpoint2021-0-31-10-57-12.ipynb
Uploading ./.ipynb_aml_checkpoints/Create_Alpha_Dataset-checkpoint2021-1-11-22-26-55.ipynb
Uploading ./.ipynb_a

--- Logging error ---
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/data/azure_storage_datastore.py", line 332, in handler
    result = future.result()
  File "/anaconda/envs/azureml_py36/lib/python3.6/concurrent/futures/_base.py", line 425, in result
    return self.__get_result()
  File "/anaconda/envs/azureml_py36/lib/python3.6/concurrent/futures/_base.py", line 384, in __get_result
    raise self._exception
  File "/anaconda/envs/azureml_py36/lib/python3.6/concurrent/futures/thread.py", line 56, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/data/azure_storage_datastore.py", line 750, in <lambda>
    lambda target, source: lambda: self.blob_service.create_blob_from_path(self.container_name, target, source)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/_vendor/azure_storage/blob/blockblobservice.py", line 463, in creat